# 高鐵查詢案例
[網站](https://www.thsrc.com.tw/tw/TimeTable/SearchResult)
## 1. Request "post" and get data

In [1]:
# Station 代號
import requests
import pandas as pd
from bs4 import BeautifulSoup
res = requests.get('https://www.thsrc.com.tw/tw/TimeTable/SearchResult')
soup = BeautifulSoup(res.text, "lxml")
name = [tag.text for tag in soup.find_all('option')[1:13]]
code = [value.attrs['value'] for value in soup.find_all('option')[1:13]]
station = pd.DataFrame({'站名':name,'代號':code},columns = ['站名','代號'])
station

,站名,代號
0,南港站,2f940836-cedc-41ef-8e28-c2336ac8fe68
1,台北站,977abb69-413a-4ccf-a109-0272c24fd490
2,板橋站,e6e26e66-7dc1-458f-b2f3-71ce65fdc95f
3,桃園站,fbd828d8-b1da-4b06-a3bd-680cdca4d2cd
4,新竹站,a7a04c89-900b-4798-95a3-c01c455622f4
5,苗栗站,e8fc2123-2aaf-46ff-ad79-51d4002a1ef3
6,台中站,3301e395-46b8-47aa-aa37-139e15708779
7,彰化站,38b8c40b-aef0-4d66-b257-da96ec51620e
8,雲林站,5f4c7bb0-c676-4e39-8d3c-f12fc188ee5f
9,嘉義站,60831846-f0e4-47f6-9b5b-46323ebdcef7


In [2]:
form_data = {
"StartStation":"977abb69-413a-4ccf-a109-0272c24fd490", # 台北
"EndStation":"9c5ac6ca-ec89-48f8-aab0-41b738cb1814", # 台南
"SearchDate":"2018/03/10",
"SearchTime":"12:00",
"SearchWay":"ArrivalInMandarin"}

response_post = requests.post("https://www.thsrc.com.tw/tw/TimeTable/SearchResult", data = form_data)
print(response_post)
response_post.text

<Response [200]>


'\r\n<!DOCTYPE html>\r\n<html lang="zh-tw">\r\n\r\n\r\n<link rel="canonical" href="https://www.thsrc.com.tw"/>\r\n\r\n<meta charset="utf-8" />\r\n<link rel="shortcut icon" href="/Content/favicon/favicon.ico" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/57_57.png" sizes="57x57" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/72_72.png" sizes="72x72" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/76_76.png" sizes="76x76" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/114_114.png" sizes="114x114" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/120_120.png" sizes="120x120" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/144_144.png" sizes="144x144" />\r\n<link rel="apple-touch-icon" href="/Content/favicon/152_152.png" sizes="152x152" />\r\n<link rel="apple-touch-icon-precomposed" href="/Content/favicon/57_57.png" sizes="57x57" />\r\n<link rel="apple-touch-icon-precomposed" href="/Content/favicon/72_72.png" sizes="72x72" />\r\

## 2. Solve html by BeautifulSoup & filter out needed tags 

In [3]:
from bs4 import BeautifulSoup
soup_post = BeautifulSoup(response_post.text, "lxml")
soup_post

<!DOCTYPE html>
<html lang="zh-tw">
<head><link href="https://www.thsrc.com.tw" rel="canonical"/>
<meta charset="utf-8"/>
<link href="/Content/favicon/favicon.ico" rel="shortcut icon"/>
<link href="/Content/favicon/57_57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/Content/favicon/72_72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/Content/favicon/76_76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/Content/favicon/114_114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/Content/favicon/120_120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/Content/favicon/144_144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/Content/favicon/152_152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/Content/favicon/57_57.png" rel="apple-touch-icon-precomposed" sizes="57x57"/>
<link href="/Content/favicon/72_72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/Content/favicon/76_76.png" rel="apple-touch-icon-p

In [4]:
soup_post.find_all("td", class_="column1")
# 找出所有td 標籤屬性class=column1 的內容，並存成list
train_number= [tag.text for tag in soup_post.find_all("td", class_="column1")]
train_number

['0609',
 '0809',
 '0613',
 '0615',
 '1209',
 '0813',
 '0619',
 '0621',
 '1213',
 '1309']

In [5]:
departure = [tag.text for tag in soup_post.find_all("td", class_="column3")]
arrival = [tag.text for tag in soup_post.find_all("td", class_="column4")]
travel_time= [tag.text for tag in soup_post.find_all("td", class_="column2")]

# 3. Reconstruct data structure by pandas

In [6]:
# 建立DataFrame，把dictionary放入，並指定columns順序
import pandas as pd
HST_df = pd.DataFrame(
{"車次":train_number,
"出發時間":departure,
"抵達時間":arrival,
"行車時間":travel_time},
columns = ["車次", "出發時間", "抵達時間", "行車時間"])

# export to csv
HST_df.to_csv("HST_schedule.csv", index = False, encoding = "cp950")
HST_df

,車次,出發時間,抵達時間,行車時間
0,0609,07:46,09:32,01:46
1,0809,08:11,10:11,02:00
2,0613,08:21,10:06,01:45
3,0615,08:46,10:32,01:46
4,1209,08:51,10:17,01:26
5,0813,09:11,11:11,02:00
6,0619,09:21,11:06,01:45
7,0621,09:46,11:32,01:46
8,1213,09:51,11:17,01:26
9,1309,10:01,11:47,01:46
